In [7]:
import string

import lyricsgenius
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
from adjustText import adjust_text
from nltk.corpus import stopwords

# API setup
# NOTE: this requires setup of Spotify and Genius developer accounts for credentials
genius = lyricsgenius.Genius('f-L9hQZOQ1VBDYt_lhkYUd-3wf0qk0fvXtwetdAWGQj-c9ICYmFx0bEwqaXNgb7cs8rtSphCVDvUUuH-rCEdKQ')

# Set up stop words and punctuation for removal
filterWords = stopwords.words("english")
translator = str.maketrans('', '', string.punctuation)

In [8]:
df_grammys = pd.read_csv('grammy_df.csv')

In [9]:
df_grammys

,title,artist,winner,year
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958
1,Catch A Falling Star,Perry Como,0,1958
2,Witchcraft (Single),Frank Sinatra,0,1958
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958
4,Fever (Single),Peggy Lee,0,1958
...,...,...,...,...
341,Woman,Doja Cat,0,2022
342,You And Me On The Rock,Brandi Carlile Featuring Lucius,0,2022
343,Good Morning Gorgeous,Mary J. Blige,0,2022
344,Easy On Me,Adele,0,2022


In [10]:
def get_lyrics_from_dataframe(df, access_token, timeout=40):
    # Initialize Genius API client with timeout
    genius = lyricsgenius.Genius(access_token, timeout=timeout)

    # Iterate over rows in dataframe
    lyrics = []
    song_names = []
    for index, row in df.iterrows():
        song_title = row['title']
        song = genius.search_song(song_title)
        if song is not None:
            song_lyrics = song.lyrics
            lyrics.append(song_lyrics)
            song_names.append(song_title)

    # Create dataframe with lyrics
    lyrics_df = pd.DataFrame(data=lyrics, index=song_names, columns=['lyrics'])
    merged_df = pd.merge(df_grammys, lyrics_df, left_on='title', right_index=True, how='left')
    merged_df.to_csv('genius_grammy.csv', index=False)
    return merged_df



In [11]:
merged_df = get_lyrics_from_dataframe(df_grammys,'ulQdQcjIRwX5BBh3pnUsTznTZpXIMbcbhWsiKhw37BdmAlkCLlfX0emCVpPSvn3Z')

Searching for "Nel Blu Dipinto Di Blu (Volare)"...
Done.
Searching for "Catch A Falling Star"...
Done.
Searching for "Witchcraft (Single)"...
Done.
Searching for "The Chipmunk Song"...
Done.
Searching for "Fever (Single)"...
Done.
Searching for "Mack The Knife"...
Done.
Searching for "The Three Bells"...
Done.
Searching for "Like Young (Single)"...
Done.
Searching for "High Hopes (Single)"...
Done.
Searching for "A Fool Such As I (Single)"...
Done.
Searching for "The Theme From A Summer Place"...
Done.
Searching for "Nice 'N Easy (Single)"...
Done.
Searching for "Mack The Knife (Single)"...
Done.
Searching for "Georgia On My Mind (Single)"...
Done.
Searching for "Are You Lonesome Tonight? (Single)"...
Done.
Searching for "Moon River"...
Done.
Searching for "Up A Lazy River"...
Done.
Searching for "The Second Time Around (Single)"...
Done.
Searching for "Take Five"...
Done.
Searching for "Big Bad John (Single)"...
Done.
Searching for "I Left My Heart In San Francisco"...
Done.
Searching

In [12]:
merged_df.dropna(inplace=True)

In [13]:
merged_df.title.nunique()

343

In [14]:
merged_df_2012 = merged_df[merged_df['year']==2012]

In [15]:
def read_lexicon(filename):
    word_sentiments = {}

    # Parse lexicon file
    with open(filename, 'r') as f:
        for line in f:
            # Ignore blank lines
            if line.strip():
                word, sentiment, value = line.split()
                if value == '1':
                    # Add sentiment to dictionary
                    cur_list = word_sentiments.get(word, [])
                    cur_list.append(sentiment)
                    word_sentiments[word] = cur_list

    return word_sentiments

In [16]:
def get_sentiment_percentages(track_df, sentiment):
    sentiment_pcts = []

    # Parse tracks
    for title in track_df.index.values:
        num_sentiment_words = 0
        all_words = str(track_df.loc[title]['lyrics']).split()

        # Parse words in lyrics
        for word in all_words:
            if sentiment in word_sentiments_dict.get(word, []):
                num_sentiment_words += 1

        sentiment_pcts.append(num_sentiment_words/len(all_words))

    return pd.DataFrame(data=sentiment_pcts, index=track_df.index.values, columns=[sentiment + '_pct'])


In [17]:
#Initialize sentiment dictionary and track dataframe
word_sentiments_dict = read_lexicon('')


#Generate sentiment percentages based on NRC Lexicon sentiments
sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
for sentiment  in sentiments:
    sentiment_df = get_sentiment_percentages(merged_df, sentiment)
    merged_df = merged_df.join(sentiment_df)

In [ ]:
merged_df

,title,artist,winner,year,lyrics,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,Nel blu dipinto di blu (Volare) Lyrics[Testo d...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Catch A Falling Star,Perry Como,0,1958,Catch a Falling Star Lyrics[Chorus]\nCatch a f...,0.000000,0.036885,0.000000,0.008197,0.045082,0.061475,0.065574,0.057377,0.000000,0.045082
2,Witchcraft (Single),Frank Sinatra,0,1958,Sweetest Maleficia Lyrics[Gilles de Rais:]\nI ...,0.020534,0.026694,0.026694,0.030801,0.014374,0.043121,0.032854,0.028747,0.010267,0.022587
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958,The Chipmunk song Lyrics[Dave:] Alright you Ch...,0.000000,0.062112,0.000000,0.000000,0.018634,0.031056,0.018634,0.024845,0.006211,0.018634
4,Fever (Single),Peggy Lee,0,1958,Jaden Smith Discography Lyrics2012\nFlame (Jus...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,Woman,Doja Cat,0,2022,TranslationsTürkçeEspañolPortuguêsItalianoΕλλη...,0.006579,0.046053,0.000000,0.000000,0.062500,0.003289,0.075658,0.006579,0.013158,0.049342
342,You And Me On The Rock,Brandi Carlile Featuring Lucius,0,2022,You and Me On The Rock Lyrics[Verse 1]\nThey b...,0.002421,0.004843,0.000000,0.002421,0.029056,0.002421,0.062954,0.000000,0.000000,0.004843
343,Good Morning Gorgeous,Mary J. Blige,0,2022,"Good Morning Gorgeous Lyrics[Intro]\nOoh\nOoh,...",0.011869,0.017804,0.008902,0.011869,0.035608,0.020772,0.041543,0.008902,0.014837,0.014837
344,Easy On Me,Adele,0,2022,TranslationsTürkçeไทย / Phasa ThaiSvenskaEspañ...,0.004425,0.044248,0.000000,0.013274,0.044248,0.008850,0.048673,0.000000,0.026549,0.013274


In [ ]:
#Rescale track_df such that gloom indices are in range [0, 100]
rescale_df = merged_df
max_gloom_value = merged_df['sadness_pct'].max()
min_gloom_value = merged_df['sadness_pct'].min()

rescale_df['gloom_index'] = (merged_df['sadness_pct'] - min_gloom_value) / (max_gloom_value - min_gloom_value) * 100

rescale_df['gloom_index'].sort_values(ascending=False)

39     100.000000
310     69.264069
324     58.298806
275     56.818182
87      55.008264
          ...    
115      0.000000
273      0.000000
106      0.000000
105      0.000000
0        0.000000
Name: gloom_index, Length: 344, dtype: float64

In [ ]:
rescale_df.head(20)

,title,artist,winner,year,lyrics,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,gloom_index
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,Nel blu dipinto di blu (Volare) Lyrics[Testo d...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Catch A Falling Star,Perry Como,0,1958,Catch a Falling Star Lyrics[Chorus]\nCatch a f...,0.000000,0.036885,0.000000,0.008197,0.045082,0.061475,0.065574,0.057377,0.000000,0.045082,54.247392
2,Witchcraft (Single),Frank Sinatra,0,1958,Sweetest Maleficia Lyrics[Gilles de Rais:]\nI ...,0.020534,0.026694,0.026694,0.030801,0.014374,0.043121,0.032854,0.028747,0.010267,0.022587,27.179391
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958,The Chipmunk song Lyrics[Dave:] Alright you Ch...,0.000000,0.062112,0.000000,0.000000,0.018634,0.031056,0.018634,0.024845,0.006211,0.018634,23.489554
4,Fever (Single),Peggy Lee,0,1958,Jaden Smith Discography Lyrics2012\nFlame (Jus...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Mack The Knife,Bobby Darin,1,1959,"Mack the Knife LyricsOh the shark, babe, has s...",0.004348,0.021739,0.000000,0.008696,0.004348,0.008696,0.013043,0.000000,0.000000,0.017391,0.000000
6,The Three Bells,The Browns,0,1959,The Three Bells LyricsThere's a village hidden...,0.004386,0.017544,0.008772,0.004386,0.035088,0.039474,0.061404,0.026316,0.013158,0.039474,24.880383
7,Like Young (Single),Andre Previn,0,1959,"Artist Archive: Tab Hunter Lyrics1956-1957: ""Y...",0.000000,0.000829,0.000000,0.000000,0.002486,0.000000,0.002486,0.000000,0.000829,0.004143,0.000000
8,High Hopes (Single),Frank Sinatra,0,1959,The Division Bell - 20th Anniversary Box Set L...,0.000000,0.004132,0.000000,0.000000,0.004132,0.000000,0.008264,0.008264,0.004132,0.004132,7.813674
9,A Fool Such As I (Single),Elvis Presley,0,1959,Amours de Voyage (Canto V) LyricsThere is a ci...,0.006795,0.016233,0.005285,0.010193,0.009060,0.019253,0.018498,0.012458,0.007173,0.012835,11.778029


In [ ]:
rescale_df.drop_duplicates(['title','artist', 'year'], inplace=True)

In [ ]:
rescale_df.sort_values(['gloom_index'], ascending=False).head(20)

,title,artist,winner,year,lyrics,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,gloom_index
39,"The \""In\"" Crowd (Single)",Ramsey Lewis Trio,0,1965,TranslationsPortuguêsWouldn’t Be a Lie LyricsM...,0.051282,0.025641,0.054487,0.060897,0.016026,0.102564,0.016026,0.105769,0.012821,0.012821,100.000000
310,Bad Guy,Billie Eilish,1,2019,TranslationsHrvatskiTürkçeSvenskaEspañolРусски...,0.054945,0.018315,0.058608,0.054945,0.018315,0.073260,0.018315,0.073260,0.007326,0.014652,69.264069
324,Colors,Black Pumas,0,2020,TranslationsEspañolFrançaisDanskColors Lyrics[...,0.000000,0.010724,0.013405,0.000000,0.013405,0.010724,0.024129,0.061662,0.005362,0.016086,58.298806
275,Stronger (What Doesn't Kill You),Kelly Clarkson,0,2012,TranslationsΕλληνικάStronger (What Doesn’t Kil...,0.019231,0.004808,0.019231,0.060096,0.007212,0.062500,0.007212,0.060096,0.009615,0.004808,56.818182
87,Rhinestone Cowboy (Single),Glen Campbell,0,1975,Finally Falling Lyrics[Verse 1]\nI used to be\...,0.014545,0.040000,0.036364,0.007273,0.054545,0.054545,0.069091,0.058182,0.040000,0.047273,55.008264
1,Catch A Falling Star,Perry Como,0,1958,Catch a Falling Star Lyrics[Chorus]\nCatch a f...,0.000000,0.036885,0.000000,0.008197,0.045082,0.061475,0.065574,0.057377,0.000000,0.045082,54.247392
160,She Drives Me Crazy (Single),Fine Young Cannibals,0,1989,She Drives Me Crazy (single remix) LyricsI can...,0.051546,0.005155,0.000000,0.056701,0.010309,0.051546,0.020619,0.051546,0.005155,0.010309,48.734770
253,Bleeding Love,Leona Lewis,0,2008,Bleeding Love Lyrics[Intro]\nHey\n\n[Verse 1]\...,0.004854,0.004854,0.033981,0.043689,0.046117,0.050971,0.065534,0.050971,0.002427,0.021845,48.190644
91,This Masquerade,George Benson,1,1976,This Masquerade Lyrics[​Intro]​\n\nAre we real...,0.034286,0.028571,0.017143,0.040000,0.005714,0.074286,0.022857,0.045714,0.000000,0.011429,43.220779
268,The Cave,Mumford & Sons,0,2011,The Cave Lyrics[Verse 1]\nIt's empty in the va...,0.015974,0.019169,0.006390,0.031949,0.022364,0.044728,0.041534,0.044728,0.009585,0.035144,42.288702
